# Notes for the Project Report

All models accuracy is under 0.5. The selected features do not work for accurate prediction.

# LOADING TRAIN AND TEST DATA

In [1]:
# Train data.
import pandas as pd
data_train = pd.read_csv("Standard_FS_pca5_train_output.csv")
data_train.shape

(97044, 6)

In [2]:
data_train.head()

,0,1,2,3,4,class
0,-0.786870,0.358685,1.037863,1.414017,-0.177119,0
1,0.100910,0.031299,0.751919,2.747381,-0.464531,0
2,8.692042,-3.114000,2.456945,0.200069,-0.325572,0
3,16.396044,-6.719431,7.398139,-6.620738,-0.488262,0
4,9.536604,-3.452310,2.937701,-0.160157,-0.415308,0


In [3]:
# split values into inpits and outputs.
values_train = data_train.values
X_train = values_train[:,0:5]
y_train = values_train[:,5]

data_train.shape

(97044, 6)

In [4]:
# Test data.
data_test = pd.read_csv("Standard_FS_pca5_test_output.csv")

# split values into inpits and outputs.
values_test = data_test.values
X_test = values_test[:,0:5]
y_test = values_test[:,5]

data_test.shape

(40158, 6)

# LOGISTIC REGRESSION

## LR with default hyperparameters

In [5]:
# Initiate the LR model with defualt hyperparameters.
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

In [6]:
# Fit the model using default hyperparameters.
# K, you don't split into train and validate sets??
lr.fit(X_train, y_train)

/Users/konstantinorlovskiy/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [7]:
# Run predictions on TEST set and see the accuracy.
lr_score = lr.score(X_test,y_test)
print(lr_score)

0.11248070123014094


In [8]:
# Build confusion matrix.
from sklearn.metrics import confusion_matrix
lr_cm = confusion_matrix(y_test, lr.predict(X_test))
print(lr_cm)

[[ 3048 17031]
 [18610  1469]]


## LR hyperparameters tuning (Random Search)

In [9]:
from sklearn.model_selection import RandomizedSearchCV

# Create array of values for tuned hyperparameters.
lr_params = {'C' : [0.01, 0.05, 0.1, 0.5, 1.0, 1.5, 2.0], 
             'max_iter' : [5, 10, 50, 100, 150, 200, 300]
             }

In [10]:
# Run random search and initiate the model with tuned parameters.
lr_random = RandomizedSearchCV(estimator=lr, param_distributions=lr_params, cv = 3, n_jobs=-1, random_state = 2019)

import time
start_time = time.time()
lr_random.fit(X_train, y_train)
finish_time = time.time()

# Summarize results
print("Best: %f using %s" % (lr_random.best_score_, lr_random.best_params_))
print("Execution time: " + str((finish_time - start_time)))

Best: 0.933793 using {'max_iter': 300, 'C': 2.0}
Execution time: 2.7015540599823


/Users/konstantinorlovskiy/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [11]:
# Apply best values of hyperparameters to the model.
lr_tuned = lr_random.best_estimator_

In [12]:
# Train the tuned model on TRAIN set and check the accuracy
lr_tuned.fit(X_train, y_train)
lr_tuned_score = lr_tuned.score(X_test,y_test)
print(lr_tuned_score)

/Users/konstantinorlovskiy/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.11258030778425221


In [13]:
# Build confusion matrix.
from sklearn.metrics import confusion_matrix
lr_tuned_cm = confusion_matrix(y_test, lr_tuned.predict(X_test))
print(lr_tuned_cm)

[[ 3052 17027]
 [18610  1469]]


## LR tuning Results

In [14]:
print("LR default hyperparameters test accuracy: ", lr_score,', parameters: ', '\n', lr.get_params())
print('Confusion matrix: ', '\n', lr_cm)
print()
print("LR tuned hyperparameters test accuracy: ", lr_tuned_score,', parameters: ', '\n', lr_tuned.get_params())
print('Confusion matrix: ', '\n', lr_tuned_cm)

LR default hyperparameters test accuracy:  0.11248070123014094 , parameters:  
 {'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'warn', 'n_jobs': None, 'penalty': 'l2', 'random_state': None, 'solver': 'warn', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}
Confusion matrix:  
 [[ 3048 17031]
 [18610  1469]]

LR tuned hyperparameters test accuracy:  0.11258030778425221 , parameters:  
 {'C': 2.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 300, 'multi_class': 'warn', 'n_jobs': None, 'penalty': 'l2', 'random_state': None, 'solver': 'warn', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}
Confusion matrix:  
 [[ 3052 17027]
 [18610  1469]]


# DECISION TREE

## DT with default hyperparameters

In [15]:
# Initiate a DT model using default hyperparameters.
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()

In [16]:
# Train model on train data.
dt.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [17]:
# Check model accuracy on the TEST set.
dt_score = dt.score(X_test, y_test)
print(dt_score)

0.49016385278151303


In [18]:
# Build confusion matrix.
from sklearn.metrics import confusion_matrix
dt_cm = confusion_matrix(y_test, dt.predict(X_test))
print(dt_cm)

[[18246  1833]
 [18641  1438]]


## DT hyperparameters tuning (Random Search)

In [19]:
# https://medium.com/@mohtedibf/indepth-parameter-tuning-for-decision-tree-6753118a03c3
# Create array of values for tuned hyperparameters.
dt_params = {'max_depth': [None, 1, 3, 5, 10, 50, 100, 300], 
             'min_samples_split': [2, 5, 10, 50, 100], 
             'min_samples_leaf': [1, 2, 5, 10, 50, 100]
            }

In [20]:
# Run random search.
from sklearn.model_selection import RandomizedSearchCV
dt_random = RandomizedSearchCV(estimator=dt, param_distributions=dt_params, cv = 10, n_jobs=-1, random_state = 2019)

import time
start_time = time.time()
dt_random.fit(X_train, y_train)
finish_time = time.time()

# Summarize results
print("Best: %f using %s" % (dt_random.best_score_, dt_random.best_params_))
print("Execution time: " + str((finish_time - start_time)))

Best: 0.981400 using {'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': None}
Execution time: 6.351583003997803


In [21]:
# Apply best values of hyperparameters to the model.
dt_tuned = dt_random.best_estimator_

In [22]:
# Train the tuned model on TRAIN set and check the accuracy
dt_tuned.fit(X_train, y_train)
dt_tuned_score = dt_tuned.score(X_test,y_test)
print(dt_tuned_score)

0.4784102793963843


In [23]:
# Build confusion matrix.
from sklearn.metrics import confusion_matrix
dt_tuned_cm = confusion_matrix(y_test, dt_tuned.predict(X_test))
print(dt_tuned_cm)

[[17745  2334]
 [18612  1467]]


## DT tuning Results

In [24]:
print("DT default hyperparameters test accuracy: ", dt_score,', parameters: ', '\n', dt.get_params())
print('Confusion matrix: ', '\n', dt_cm)
print()
print("DT tuned hyperparameters test accuracy: ", dt_tuned_score,', parameters: ', '\n', dt_tuned.get_params())
print('Confusion matrix: ', '\n', dt_tuned_cm)

DT default hyperparameters test accuracy:  0.49016385278151303 , parameters:  
 {'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'presort': False, 'random_state': None, 'splitter': 'best'}
Confusion matrix:  
 [[18246  1833]
 [18641  1438]]

DT tuned hyperparameters test accuracy:  0.4784102793963843 , parameters:  
 {'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'presort': False, 'random_state': None, 'splitter': 'best'}
Confusion matrix:  
 [[17745  2334]
 [18612  1467]]


# RANDOM FOREST

## RF with default hyperparameters

In [25]:
# Initiate a RF model using default hyperparameters.
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

In [26]:
# Train model on train data.
rf.fit(X_train, y_train)

/Users/konstantinorlovskiy/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [27]:
# Check model accuracy on the TEST set.
rf_score = rf.score(X_test, y_test)
print(rf_score)

0.28738981024951443


In [28]:
# Build confusion matrix.
from sklearn.metrics import confusion_matrix
rf_cm = confusion_matrix(y_test, rf.predict(X_test))
print(rf_cm)

[[10072 10007]
 [18610  1469]]


## RF hyperparameters tuning (Random Search)

In [29]:
# Define a grid of hyperparameters.
rf_params = { 'n_estimators': [1, 5, 10, 30, 50, 100, 200, 500], 
             'max_depth': [None, 1, 2, 4, 8, 20, 50, 100], 
             'min_samples_leaf': [1, 5, 10, 50, 100], 
             'max_features': [None, 'auto', 'log2']
            }

In [30]:
# Run random search.
from sklearn.model_selection import RandomizedSearchCV
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=rf_params, cv = 3, n_jobs=-1, random_state = 2019)

import time
start_time = time.time()
rf_random.fit(X_train, y_train)
finish_time = time.time()

# Summarize results
print("Best: %f using %s" % (rf_random.best_score_, rf_random.best_params_))
print("Execution time: " + str((finish_time - start_time)))

Best: 0.974960 using {'n_estimators': 50, 'min_samples_leaf': 100, 'max_features': 'auto', 'max_depth': 4}
Execution time: 23.442732095718384


In [31]:
# Apply best values of hyperparameters to the model.
rf_tuned = rf_random.best_estimator_

In [32]:
# Train the tuned model on TRAIN set and check the accuracy
rf_tuned.fit(X_train, y_train)
rf_tuned_score = rf_tuned.score(X_test,y_test)
print(rf_tuned_score)

0.4496488868967578


In [33]:
# Build confusion matrix.
rf_tuned_cm = confusion_matrix(y_test, rf_tuned.predict(X_test))
print(rf_tuned_cm)

[[18057  2022]
 [20079     0]]


## RF tuning Results

In [34]:
print("RF default hyperparameters test accuracy: ", rf_score,', parameters: ', '\n', rf.get_params())
print('Confusion matrix: ', '\n', rf_cm)
print()
print("RF tuned hyperparameters test accuracy: ", rf_tuned_score,', parameters: ', '\n', rf_tuned.get_params())
print('Confusion matrix: ', '\n', rf_tuned_cm)

RF default hyperparameters test accuracy:  0.28738981024951443 , parameters:  
 {'bootstrap': True, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 10, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}
Confusion matrix:  
 [[10072 10007]
 [18610  1469]]

RF tuned hyperparameters test accuracy:  0.4496488868967578 , parameters:  
 {'bootstrap': True, 'class_weight': None, 'criterion': 'gini', 'max_depth': 4, 'max_features': 'auto', 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 100, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 50, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}
Confusion matrix:  
 [[18057 

# SVM (SVC)

## SVC with default hyperparameters

In [35]:
from sklearn import svm
svc = svm.SVC()

In [36]:
svc.fit(X_train, y_train)

/Users/konstantinorlovskiy/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [37]:
svc_score = svc.score(X_test, y_test)
print(svc_score)

0.4807012301409433


In [38]:
# Build confusion matrix.
from sklearn.metrics import confusion_matrix
svc_cm = confusion_matrix(y_test, svc.predict(X_test))
print(svc_cm)

[[19304   775]
 [20079     0]]


 ## SVC hyperparameters tuning (Random Search)

In [39]:
# Define a grid of hyperparameters.
svc_params = { 'C': [0.1, 0.5, 1, 3, 5], 
             'gamma': ['scale', 'auto', 0.01, 0.1, 1, 10]
            }

In [40]:
# Run random search.
from sklearn.model_selection import RandomizedSearchCV
svc_random = RandomizedSearchCV(estimator=svc, n_iter=3, param_distributions=svc_params, cv = 3, n_jobs=-1, 
                                random_state = 2019)

import time
start_time = time.time()
svc_random.fit(X_train, y_train)
finish_time = time.time()

# Summarize results
print("Best: %f using %s" % (svc_random.best_score_, svc_random.best_params_))
print("Execution time: " + str((finish_time - start_time)))

Best: 0.975516 using {'gamma': 0.1, 'C': 3}
Execution time: 31.995261907577515


In [41]:
# Apply best values of hyperparameters to the model.
svc_tuned = svc_random.best_estimator_

In [42]:
# Train the tuned model on TRAIN set and check the accuracy
svc_tuned.fit(X_train, y_train)
svc_tuned_score = svc_tuned.score(X_test,y_test)
print(svc_tuned_score)

0.44613775586433585


In [43]:
# Build confusion matrix.
from sklearn.metrics import confusion_matrix
svc_tuned_cm = confusion_matrix(y_test, svc_tuned.predict(X_test))
print(svc_tuned_cm)

[[17916  2163]
 [20079     0]]


## SVC tuning Results

In [44]:
print("SVC default hyperparameters test accuracy: ", svc_score, 
      ', parameters: ', '\n', svc.get_params())
print('Confusion matrix: ', '\n', svc_cm)
print()
print("SVC tuned hyperparameters test accuracy: ", svc_tuned_score, 
      ', parameters: ', '\n', svc_tuned.get_params())
print('Confusion matrix: ', '\n', svc_tuned_cm)

SVC default hyperparameters test accuracy:  0.4807012301409433 , parameters:  
 {'C': 1.0, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 'auto_deprecated', 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
Confusion matrix:  
 [[19304   775]
 [20079     0]]

SVC tuned hyperparameters test accuracy:  0.44613775586433585 , parameters:  
 {'C': 3, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.1, 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
Confusion matrix:  
 [[17916  2163]
 [20079     0]]


# KNN classifier

## KNN with default hyperparameters

In [45]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()

In [46]:
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [47]:
knn_score = knn.score(X_test, y_test)
print(knn_score)

0.33552467752378107


In [48]:
# Build confusion matrix.
from sklearn.metrics import confusion_matrix
knn_cm = confusion_matrix(y_test, knn.predict(X_test))
print(knn_cm)

[[12002  8077]
 [18607  1472]]


## KNN hyperparameters tuning (Random Search)¶

In [49]:
# Define a grid of hyperparameters.
knn_params = {'n_neighbors': [3, 5, 10, 20, 50, 100], 
              'weights': ['uniform', 'distance'], 
              'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'], 
              'leaf_size': [10, 30, 50, 100], 
              'p': [1, 2]
            }

In [50]:
# Run random search.
from sklearn.model_selection import RandomizedSearchCV
knn_random = RandomizedSearchCV(estimator=knn, n_iter=3, param_distributions=knn_params, cv = 3, n_jobs=-1, 
                                random_state = 2019)

import time
start_time = time.time()
knn_random.fit(X_train, y_train)
finish_time = time.time()

# Summarize results
print("Best: %f using %s" % (knn_random.best_score_, knn_random.best_params_))
print("Execution time: " + str((finish_time - start_time)))

Best: 0.978463 using {'weights': 'uniform', 'p': 2, 'n_neighbors': 10, 'leaf_size': 100, 'algorithm': 'brute'}
Execution time: 49.99443030357361


In [51]:
# Apply best values of hyperparameters to the model.
knn_tuned = knn_random.best_estimator_

In [52]:
# Train the tuned model on TRAIN set and check the accuracy
knn_tuned.fit(X_train, y_train)
knn_tuned_score = knn_tuned.score(X_test,y_test)
print(knn_tuned_score)

0.3359978086558095


In [53]:
# Build confusion matrix.
from sklearn.metrics import confusion_matrix
knn_tuned_cm = confusion_matrix(y_test, knn_tuned.predict(X_test))
print(knn_tuned_cm)

[[12020  8059]
 [18606  1473]]


## KNN tuning Results

In [54]:
print("KNN default hyperparameters test accuracy: ", knn_score, 
      ', parameters: ', '\n', knn.get_params())
print('Confusion matrix: ', '\n', knn_cm)
print()
print("KNN tuned hyperparameters test accuracy: ", knn_tuned_score, 
      ', parameters: ', '\n', knn_tuned.get_params())
print('Confusion matrix: ', '\n', knn_tuned_cm)

KNN default hyperparameters test accuracy:  0.33552467752378107 , parameters:  
 {'algorithm': 'auto', 'leaf_size': 30, 'metric': 'minkowski', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 5, 'p': 2, 'weights': 'uniform'}
Confusion matrix:  
 [[12002  8077]
 [18607  1472]]

KNN tuned hyperparameters test accuracy:  0.3359978086558095 , parameters:  
 {'algorithm': 'brute', 'leaf_size': 100, 'metric': 'minkowski', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 10, 'p': 2, 'weights': 'uniform'}
Confusion matrix:  
 [[12020  8059]
 [18606  1473]]


# Naive Bayes Classifier (NBC)

In [55]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
nbc = GaussianNB()

In [56]:
# Fit the model and display score
nbc.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [57]:
# Run predictions on TEST set and see the accuracy.
nbc_score = nbc.score(X_test,y_test)
print(nbc_score)

0.48767368892873153


In [58]:
# Build confusion matrix.
from sklearn.metrics import confusion_matrix
nbc_cm = confusion_matrix(y_test, nbc.predict(X_test))
print(nbc_cm)

[[19584   495]
 [20079     0]]


## NBC hyperparameters tuning (Random Search)

In [59]:
# Define a grid of hyperparameters.
nbc_params = {'var_smoothing': [1e-12, 1e-10, 1e-09, 1e-05, 1e-04, 1e-03, 1e-02, 1e-01, 1]
            }

In [60]:
# Run random search.
from sklearn.model_selection import RandomizedSearchCV
nbc_random = RandomizedSearchCV(estimator=nbc, n_iter=3, param_distributions=nbc_params, cv = 3, n_jobs=-1, 
                                random_state = 2019)

import time
start_time = time.time()
nbc_random.fit(X_train, y_train)
finish_time = time.time()

# Summarize results
print("Best: %f using %s" % (nbc_random.best_score_, nbc_random.best_params_))
print("Execution time: " + str((finish_time - start_time)))

Best: 0.934051 using {'var_smoothing': 1e-05}
Execution time: 0.13223481178283691


In [61]:
# Apply best values of hyperparameters to the model.
nbc_tuned = nbc_random.best_estimator_

In [62]:
# Train the tuned model on TRAIN set and check the accuracy
nbc_tuned.fit(X_train, y_train)
nbc_tuned_score = nbc_tuned.score(X_test,y_test)
print(nbc_tuned_score)

0.4876487872902037


In [63]:
# Build confusion matrix.
from sklearn.metrics import confusion_matrix
nbc_tuned_cm = confusion_matrix(y_test, nbc_tuned.predict(X_test))
print(nbc_tuned_cm)

[[19583   496]
 [20079     0]]


## NBC tuning Results

In [64]:
print("NBC default hyperparameters test accuracy: ", nbc_score, 
      ', parameters: ', '\n', nbc.get_params())
print('Confusion matrix: ', '\n', nbc_cm)
print()
print("NBC tuned hyperparameters test accuracy: ", nbc_tuned_score, 
      ', parameters: ', '\n', nbc_tuned.get_params())
print('Confusion matrix: ', '\n', nbc_tuned_cm)

NBC default hyperparameters test accuracy:  0.48767368892873153 , parameters:  
 {'priors': None, 'var_smoothing': 1e-09}
Confusion matrix:  
 [[19584   495]
 [20079     0]]

NBC tuned hyperparameters test accuracy:  0.4876487872902037 , parameters:  
 {'priors': None, 'var_smoothing': 1e-05}
Confusion matrix:  
 [[19583   496]
 [20079     0]]


# Compare Algorithms Performance

In [66]:
print("LR tuned hyperparameters test accuracy: ", lr_tuned_score)
print('Confusion matrix: ', '\n', lr_tuned_cm, '\n')

print("DT tuned hyperparameters test accuracy: ", dt_tuned_score)
print('Confusion matrix: ', '\n', dt_tuned_cm, '\n')

print("RF tuned hyperparameters test accuracy: ", rf_tuned_score)
print('Confusion matrix: ', '\n', rf_tuned_cm, '\n')

print("SVC tuned hyperparameters test accuracy: ", svc_tuned_score)
print('Confusion matrix: ', '\n', svc_tuned_cm, '\n')

print("KNN tuned hyperparameters test accuracy: ", knn_tuned_score)
print('Confusion matrix: ', '\n', knn_tuned_cm, '\n')

print("NBC tuned hyperparameters test accuracy: ", nbc_tuned_score)
print('Confusion matrix: ', '\n', nbc_tuned_cm)

LR tuned hyperparameters test accuracy:  0.11258030778425221
Confusion matrix:  
 [[ 3052 17027]
 [18610  1469]] 

DT tuned hyperparameters test accuracy:  0.4784102793963843
Confusion matrix:  
 [[17745  2334]
 [18612  1467]] 

RF tuned hyperparameters test accuracy:  0.4496488868967578
Confusion matrix:  
 [[18057  2022]
 [20079     0]] 

SVC tuned hyperparameters test accuracy:  0.44613775586433585
Confusion matrix:  
 [[17916  2163]
 [20079     0]] 

KNN tuned hyperparameters test accuracy:  0.3359978086558095
Confusion matrix:  
 [[12020  8059]
 [18606  1473]] 

NBC tuned hyperparameters test accuracy:  0.4876487872902037
Confusion matrix:  
 [[19583   496]
 [20079     0]]
